## 자영업자는 농수축산물 일자별 도소매 가격으로 어떤 인사이트를 얻을 수 있을까요?

* 가격 변동 원인: 특정 품목의 가격이 연중 어느 시기에 가장 높고 낮은지, 그 원인은 무엇일까요?
* 지역 차이 요인: 어떤 지역에서 특정 품목의 가격이 가장 높고 낮은지, 그 이유는 무엇일까요?
* 최적 할인 전략: 어떤 품목에서 할인 정책이 가장 효과적이며, 최적의 할인 전략(할인 폭, 기간 등)은 무엇일까요?
* 품종/등급 선호도: 어떤 품종이 가장 높은 가격 프리미엄을 가지며, 소비자들의 품종 및 등급 선호도는 어떻게 변화하고 있을까요?
* 유통 효율화 방안: 어떤 품목에서 도소매 가격 차이가 가장 크고 작은지, 유통 마진을 줄이기 위한 효과적인 방안은 무엇일까요?

## 농수축산물 일자별 도소매 가격

https://kadx.co.kr/opmk/frn/pmumkproductDetail/PMU_6faffdb1-c1c0-4c1e-aacb-9eb34b32f772/5

In [ ]:
# !pip install -Uq koreanize-matplotlib

In [ ]:
import pandas as pd
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# 데이터 컬럼 정보를 불러옵니다.|
df_col = pd.read_csv("data/kadx-utf-8-at-product-price-detail.csv")
df_col.shape

In [ ]:
df_col

In [ ]:
# 영문 컬럼명을 한글로 변환하기 위한 매핑 딕셔너리를 생성합니다.
col_dict = df_col[["nameEn","nameKo"]].set_index("nameEn")['nameKo'].to_dict()
col_dict

In [ ]:
from glob import glob

file_list = glob("data/daily-price/*.csv")
file_list

In [ ]:
df_list = []
for file_path in file_list:
    df_temp = pd.read_csv(file_path, encoding='cp949')
    df_list.append(df_temp)

In [ ]:
raw = pd.concat(df_list)
raw.shape

In [ ]:
raw.head()

In [ ]:
# 원본 데이터의 컬럼명을 한글로 변환합니다.
df = raw.rename(columns=col_dict)

In [ ]:
# 변환된 데이터의 첫 몇 줄을 확인합니다.
df.head()

## 기초 기술통계

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.hist(figsize=(10, 10), bins=50);

In [ ]:
df.describe(exclude="number")

### 유일값 분석

In [ ]:
df.nunique()

In [ ]:
nuniq = df.nunique() 
nuniq[nuniq > 1]

## 데이터 타입 변경

In [ ]:
df["가격등록일자"] = pd.to_datetime(df["가격등록일자"].astype(str))

## 빈도 분석

In [ ]:
df["품종명"].value_counts()

## 범주별 가격 분석

In [ ]:
df.columns

In [ ]:
pd.crosstab(df['조사구분명'], df['산물등급명'])

In [ ]:
plt.figure(figsize=(8, 3))
sns.barplot(data=df, x='산물등급명', y='품목가격')
plt.show()

In [ ]:
plt.figure(figsize=(16, 3))
sns.barplot(data=df, x='품목명', y='품목가격')
plt.show()

In [ ]:
plt.figure(figsize=(16, 3))
sns.barplot(data=df, x='품목명', y='품목가격', hue="할인가격여부")
plt.show()

In [ ]:
pd.crosstab(df["할인가격여부"], df["품목명"])

In [ ]:
df.groupby(["품목명", "할인가격여부"])["품목가격"].mean().unstack().style.format("{:,.0f}")

In [ ]:
plt.figure(figsize=(8, 3))
sns.barplot(data=df, x='산물등급명', y='품목가격')
plt.show()

In [ ]:
plt.figure(figsize=(8, 3))
sns.barplot(data=df, x='산물등급명', y='품목가격', hue="할인가격여부")
plt.show()

## 상관 분석

In [ ]:
corr = df.corr(numeric_only=True).dropna(axis=0, how="all").dropna(axis=1, how="all")
corr

In [ ]:
mask = np.triu(np.ones_like(corr))

In [ ]:
sns.heatmap(corr, annot=True, fmt=".2f", mask=mask, cmap="seismic")

## 추가로 해볼 수 있는 것
* 시계열 분석: 월별, 계절별 가격 변동 추이를 시각화하고 품목별로 비교합니다.
* 지리적 분석: 시도별, 시군구별 평균 가격을 비교하고 지도 시각화를 수행합니다.
* 할인 효과 모델링: 할인 여부에 따른 판매량 변화를 시계열로 분석하고, 할인 폭과 판매량 증가 사이의 관계를 모델링합니다.
* 프리미엄 계산: 품종별, 등급별 평균 가격을 비교하고 프리미엄을 계산합니다.
* 유통 마진 분석: 도매가와 소매가의 차이를 품목별, 시기별로 비교하고, 유통 단계별 가격 변동의 시차 효과를 분석합니다.